In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')

In [ ]:
df.sample(5)

In [ ]:
df.shape

In [ ]:
# steps -> Data Cleaning -> EDA -> Text Preprocessing -> Model Building -> Evaluation -> Improvement -> Website -> Deploy

## 1. Data Cleaning

In [ ]:
df.info()

In [ ]:
# so here as last three columns are null mostly so we drop them

In [ ]:
df.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],inplace = True)

In [ ]:
df.sample(5)

In [ ]:
# renaming the columns
df.rename(columns = {'v1': 'target','v2':'text'}, inplace = True)

In [ ]:
df.sample(5)

In [ ]:
# 9:41

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()

In [ ]:
encoder.fit_transform(df['target'])

In [ ]:
df['target'] = encoder.fit_transform(df['target'])

In [ ]:
df.sample(5)

In [ ]:
# missing values
df.isnull().sum()

In [ ]:
# check for duplicated values
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates(keep = 'first')

In [ ]:
df.head()

In [ ]:
df.duplicated().sum()

In [ ]:
df.shape

## 2. EDA(Exploratery data analysis)

In [ ]:
df.value_counts()

In [ ]:
df['target'].value_counts()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.pie(df['target'].value_counts(), labels = ['ham','spam'], autopct = "%0.2f")
plt.show()

In [ ]:
# in above data is imbalanced

In [ ]:
# nltk is natural language tool kit and it is used for deeper analysis
import nltk
nltk.download('punkt')

In [ ]:
df['text']

In [ ]:
# this will give me the numbers of characters in one row sentence
df['text'].apply(len)

In [ ]:
df['num_characters'] = df['text'].apply(len)

In [ ]:
df.head()

In [ ]:
# number of words
df['text'].apply(lambda x:nltk.word_tokenize(x))

In [ ]:
df['text'].apply(lambda x:len(nltk.word_tokenize(x)))

In [ ]:
df['num_words'] = df['text'].apply(lambda x:len(nltk.word_tokenize(x)))

In [ ]:
df.head()

In [ ]:
df['num_sentences'] = df['text'].apply(lambda x:len(nltk.sent_tokenize(x)))

In [ ]:
df.head()

In [ ]:
df[['num_characters','num_words','num_sentences']].describe()

In [ ]:
# for ham messages
df[df['target'] == 0][['num_characters','num_words','num_sentences']].describe()

In [ ]:
# for spam messages
df[df['target'] == 1][['num_characters','num_words','num_sentences']].describe()

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize = (12,8))
sns.histplot(df[df['target'] == 0]['num_characters'])
sns.histplot(df[df['target'] == 1]['num_characters'], color = 'red')


In [ ]:
plt.figure(figsize = (12,8))
sns.histplot(df[df['target'] == 0]['num_s'])
sns.histplot(df[df['target'] == 1]['num_s'], color = 'red')


In [ ]:
sns.pairplot(df,hue = 'target')

In [ ]:
sns.heatmap(df.corr())

In [ ]:
sns.heatmap(df.select_dtypes(include=['number']).corr(), annot = True)

In [ ]:
df.corr()

In [ ]:
df.select_dtypes(include=['number']).corr()

## 3. Data Preprocessing

In [ ]:
def transform_text(text):
    text = text.lower()
    return text

In [ ]:
transform_text('Hello How are You')

In [ ]:
def transform_text1(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    return text

In [ ]:
# after adding word_tokenize to the function the output is 
transform_text1('Hello How are You')

In [ ]:
df['text'][0]

In [ ]:
# to elemenate special cheracters we use
def transform_text3(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
    return y

In [ ]:
# this will elemenate % and %% from below
transform_text3('Hello How are You 20% %% eg')

In [ ]:
from nltk.corpus import stopwords
stopwords.words('english')

In [ ]:
# the solution for above problem is as
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stopwords.words('english')

In [ ]:
import string
string.punctuation

In [ ]:
# to elemenate stopwords we use
def transform_text4(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
            
    text = y[:]
    y.clear()
    
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    return y

In [ ]:
# here all the words that are not contributing to the meaning of the sentence are elemenated 
transform_text4('Hello How are You 20% %% eg')

In [ ]:
transform_text4('Did you like my presentation on ML?')

In [ ]:
# this function is used to get words in their root words
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
ps.stem('dancing')

In [ ]:
# to elemenate stopwords we use
def transform_text5(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
            
    text = y[:]
    y.clear()
    
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
            
    text = y[:]
    y.clear()
    
    for i in text:
        y.append(ps.stem(i))
            
    return " ".join(y)

In [ ]:
transform_text5('I Loved the YT Lectures on Machine Learniong, How about you?')

In [ ]:
df['text'][0]

In [ ]:
transform_text5('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...')

In [ ]:
df['text'].apply(transform_text5)

In [ ]:
df['transformed_text'] = df['text'].apply(transform_text5)

In [ ]:
df.head()

In [ ]:
pip install wordcloud

In [ ]:
from wordcloud import WordCloud
wc = WordCloud(width = 50,height = 50, min_font_size = 10, background_color = 'black')

In [ ]:
wc.generate(df[df['target'] == 1]['transformed_text'].str.cat(sep = " "))

In [ ]:
wc = WordCloud(font_path='/Library/Fonts/df',
               width=50, height=50, min_font_size=10, background_color='black')


In [ ]:
wc.generate(df[df['target'] == 1]['transformed_text'].str.cat(sep = " "))